In [18]:
#program for scraping Stardew Valey wiki page, creating xls file.

import requests
from bs4 import BeautifulSoup
import pandas as pd

data = requests.get('https://stardewvalleywiki.com/Cooking#Recipes')

soup = BeautifulSoup(data.text, 'html.parser')

dish = []
ingridients = []
energy_health =[]
extra = []
price = []

recipes = soup.find('table', {'class':'wikitable sortable'})
trs = recipes.find_all('tr')

for tr in trs:
    tds = tr.find_all('td')
    if len(tds)==9 or len(tds)==12:    #some tr have 9 tds and some 12
        name = tds[1].text.strip()
        dish.append(name)
        ingr = tds[3].text.strip()
        ingridients.append(ingr)
        ener_heal = tds[4].text.strip() 
        energy_health.append(ener_heal)
        buff = tds[5].text.strip() 
        extra.append(buff)
    if len(tds)==9:
        gold = tds[8].text.strip()  
        price.append(gold)
    if len(tds)==12:
        gold2 = tds[11].text.strip()
        price.append(gold2)
    

In [45]:
#creating a DataFrame 
df = pd.DataFrame({'dish': dish,
                   'ingridients': ingridients,
                   'energy_health': energy_health,
                   'buff': extra,
                   'price': price})

In [46]:
#splitting energy and health column to 2 columns, as integers
df["energy"] = df.energy_health.apply(lambda str: str.split()[0]).astype('int64')
df["health"] = df.energy_health.apply(lambda str: str.split()[1]).astype('int64')

In [47]:
del df['energy_health']

In [48]:
#removing 'g' from price column, making integer type
df['price'] = df.price.apply(lambda str: str.replace('g', '')).astype('int64')
df.head()

,dish,ingridients,buff,price,energy,health
0,Fried Egg,Egg (1),N/A,35,50,22
1,Omelet,Egg (1) Milk (1),N/A,125,100,45
2,Salad,Leek (1) Dandelion (1) Vinegar (1),N/A,110,113,50
3,Cheese Cauliflower,Cauliflower (1) Cheese (1),N/A,300,138,62
4,Baked Fish,Sunfish (1) Bream (1) Wheat Flour (1),N/A,100,75,33


In [49]:
df.dtypes

dish           object
ingridients    object
buff           object
price           int64
energy          int64
health          int64
dtype: object

In [50]:
#saving as csv file
df.to_csv("stardew_recipes.csv", index=False)

In [51]:
#saving as Excel file
df.to_excel('stardew_recipes.xlsx', index=False)